In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("..")

import chess
import numpy as np
import board_representation.sentimate as br
import torch
from data_modules.move_as_tensor import MoveAsTensorDatasetLazy
from data_modules.lightning import MoveEvaluationDataModule

from utils.utils import load_sql_to_df

import torch
import models.move_evaluation as me_models
import models.lightning as pl_models
import pytorch_lightning as pl

from datasets import Dataset
from torch.utils.data import DataLoader
from data_modules.lightning import MoveEvaluationDataModule
from sklearn.model_selection import train_test_split

from pytorch_lightning.loggers import TensorBoardLogger, CSVLogger
from pytorch_lightning.callbacks import RichProgressBar

/home/kamil/miniconda3/envs/python/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
2024-02-23 13:22:14.995506: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-23 13:22:14.995544: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-23 13:22:14.996563: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-23 13:22:15.002490: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow

In [3]:
db_filename = "./../../data/databases/chess_moves_quality.db"
moves_df = load_sql_to_df("SELECT * FROM sentimate_v1", db_filename)
moves_df = moves_df.rename(columns={"fen": "position"})

In [4]:
data_module = MoveEvaluationDataModule(dataframe=moves_df, batch_size=1024, num_workers=15)

In [5]:
data_module.setup()

Train size: 10978294, Validation size: 110892


In [18]:
model = me_models.SentimateNet(output_size=2, dropout=0.1, skip_connection=True, input_channels=24)
# model = me_models.AlphaZeroOnlyValueHead(num_blocks=2, output_size=2)
pl_model = pl_models.PLClassifierModel(model, learning_rate=0.0015, save_dir="../../models/move_evaluation/sentimate/sentimate-2-22/no_skip_connection")

In [19]:
tensorboard_logger = TensorBoardLogger(save_dir="../../lightning_logs/sentimate/no_skip_connection", name=f"{type(pl_model.model).__name__}")

trainer = pl.Trainer(
    accelerator="gpu",
    max_epochs=3,
    callbacks=[RichProgressBar()],
    logger=tensorboard_logger,
    # default_root_dir=
    # fast_dev_run=True
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [20]:
trainer.fit(
    model=pl_model,
    datamodule=data_module,
    # ckpt_path="../../lightning_logs/sentimate/SentimateNet/version_0/checkpoints/epoch=49-step=536050.ckpt"
    # ckpt_path="../../lightning_logs/sentimate/tensorboard/SentimateNet/version_1/checkpoints/epoch=29-step=321630.ckpt"
)

Missing logger folder: ../../lightning_logs/sentimate/no_skip_connection/SentimateNet
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name            ┃ Type               ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ model           │ SentimateNet       │  888 K │
│ 1 │ train_acc       │ MulticlassAccuracy │      0 │
│ 2 │ valid_acc_micro │ MulticlassAccuracy │      0 │
│ 3 │ valid_acc_macro │ MulticlassAccuracy │      0 │
│ 4 │ test_acc        │ MulticlassAccuracy │      0 │
└───┴─────────────────┴────────────────────┴────────┘

Trainable params: 888 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 888 K                                                                                                
Total estimated model params size (MB): 3

Output()

`Trainer.fit` stopped: `max_epochs=3` reached.
